<a href="https://colab.research.google.com/github/Daniel-Yao-Chengdu/NLP-project/blob/master/Pretraining/Continue_training_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* A very good website for learning trainer: [here](https://zhuanlan.zhihu.com/p/363670628)
* Reference for this code: [here](https://www.kaggle.com/code/arnabs007/pretrain-a-bert-language-model-from-scratch/notebook) 

In [ ]:
#!pip install transformers
import transformers
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling, BertTokenizer
import torch

In [44]:
# import data and process the data into token numbers
# only choose sentences whose weights are higher than 0.75, then the number of sentences is 61841
dataset=torch.load('corpus 4.pt')
weight=torch.load('corpurs_4_weights.pt')

new_dataset=[i for i,j in zip(dataset,weight) if j>=0.75]
new_weight=[j for j in weight if j>=0.75]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dataset_id=tokenizer(new_dataset,max_length=512,truncation=True)['input_ids']

#Use data_collator to create pairs of input data and labels
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
# Define model
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling, BertTokenizer
 
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

In [47]:
# Deine training arguments
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='my_corpus_training',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps= 50, # for save the models
    save_total_limit=2,
    gradient_accumulation_steps=64,
    prediction_loss_only=True,
    logging_steps=50, #for printing loss
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset_id,
)

In [ ]:
%%time
trainer.train()
trainer.save_model('my_corpus_training')